In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.models import load_model
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import time

# Training Data

In [2]:
train_data = pd.read_csv("csv_files/train.csv")
train_data

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
0,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,150,1.20,45.78
1,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,160,1.12,39.00
2,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,170,1.06,32.78
3,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,180,1.00,27.17
4,0.16,1.78,0.61,0.06,-0.62,1.0,180.00,190,0.95,22.18
...,...,...,...,...,...,...,...,...,...,...
1748995,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,200,0.86,25.49
1748996,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,210,0.82,22.21
1748997,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,220,0.78,19.28
1748998,0.34,2.23,0.55,0.14,-0.83,1.0,171.41,230,0.75,16.67


In [3]:
train_data.describe()

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
count,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06,1.749000e+06
mean,2.770000e-01,1.710900e+00,3.634333e-01,9.103333e-02,-5.388667e-01,1.000000e+00,1.795318e+02,1.960000e+02,9.384750e-01,2.772866e+01
std,1.372941e-01,7.905263e-01,2.244740e-01,3.526943e-02,2.017475e-01,4.316752e-01,1.251597e+01,3.039738e+01,1.608779e-01,1.714504e+01
min,1.000000e-02,3.000000e-02,1.000000e-02,3.000000e-02,-9.000000e-01,0.000000e+00,1.500100e+02,1.500000e+02,6.000000e-01,-0.000000e+00
25%,1.600000e-01,1.097500e+00,1.700000e-01,6.000000e-02,-7.100000e-01,6.900000e-01,1.704700e+02,1.700000e+02,8.100000e-01,1.399000e+01
50%,2.800000e-01,1.765000e+00,3.400000e-01,9.000000e-02,-5.400000e-01,1.000000e+00,1.796900e+02,1.950000e+02,9.200000e-01,2.619000e+01
75%,4.000000e-01,2.392500e+00,5.525000e-01,1.200000e-01,-3.700000e-01,1.310000e+00,1.885400e+02,2.200000e+02,1.060000e+00,3.973000e+01
max,5.000000e-01,3.000000e+00,7.900000e-01,1.500000e-01,-2.000000e-01,2.000000e+00,2.099700e+02,2.500000e+02,1.400000e+00,1.006500e+02


In [4]:
# Obtain features and label -> X, Y
X, Y = train_data.drop(["moneyness", "price"], axis=1), train_data["price"]

In [5]:
# Normalize the data -> X_normalized, Y_normalized
## Normalize the model parameters in a domain
def custom_min_max_normalization(x, xmin, xmax):
    return (2 * x - (xmax + xmin)) / (xmax - xmin)

model_parameters = X.drop(["tau", "stockPrice", "strike"], axis=1)
option_properties = X.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)

min_vals = pd.Series({ 
    'varsigma': 0.01,
    'kappa': 0,
    'v0': 0.03,
    'delta': 0.01,
    'rho': -0.9
})

max_vals = pd.Series({
    'varsigma': 0.5,
    'kappa': 3.0,
    'v0': 0.15,
    'delta': 0.8,
    'rho': -0.2
})

normalized_model_parameters = pd.DataFrame()
for column in model_parameters.columns:
    normalized_model_parameters[column] = custom_min_max_normalization(
        model_parameters[column], 
        min_vals[column], 
        max_vals[column]
    )

scaler = MinMaxScaler() 
normalized_option_properties = scaler.fit_transform(option_properties)
normalized_option_properties = pd.DataFrame(normalized_option_properties, columns=option_properties.columns)

X_normalized = pd.concat([normalized_model_parameters, normalized_option_properties], axis=1).values

Y_normalized = scaler.fit_transform(Y.values.reshape(-1, 1))
Y_normalized = pd.Series(Y_normalized.flatten(), name=Y.name).values

In [6]:
# Define sequences of days
num_features = X.shape[1]
num_days = 53 # number of days
num_samples = len(X)
num_samples_per_day = num_samples // num_days

X_days = [X_normalized[i*num_samples_per_day : (i+1)*num_samples_per_day] for i in range(num_days)]
Y_days = [Y_normalized[i*num_samples_per_day : (i+1)*num_samples_per_day] for i in range(num_days)]

X_seq = np.stack(X_days, axis=1)
Y_seq = np.stack(Y_days, axis=1)

# Test Data

In [7]:
test = pd.read_csv("csv_files/outOfSample.csv")
test = round(test,2)

In [8]:
test.describe()

,varsigma,kappa,delta,v0,rho,tau,stockPrice,strike,moneyness,price
count,42400.000000,42400.00000,42400.000000,42400.00000,42400.000000,42400.000000,42400.000000,42400.000000,42400.000000,42400.000000
mean,0.300600,1.72840,0.346600,0.09240,-0.525600,1.100000,176.103230,190.000000,0.939944,30.159162
std,0.130192,0.76819,0.225335,0.03592,0.194786,0.422904,10.120805,22.360943,0.124657,15.160794
min,0.040000,0.10000,0.010000,0.03000,-0.870000,0.250000,155.110000,160.000000,0.700000,0.010000
25%,0.210000,1.23000,0.150000,0.06000,-0.700000,0.805000,168.540000,175.000000,0.840000,18.660000
50%,0.305000,1.80000,0.310000,0.09000,-0.520000,1.100000,175.160000,190.000000,0.930000,29.560000
75%,0.420000,2.28000,0.520000,0.12000,-0.350000,1.395000,183.490000,205.000000,1.040000,40.860000
max,0.490000,2.99000,0.780000,0.15000,-0.210000,1.950000,199.870000,220.000000,1.250000,81.000000


In [9]:
TestX, TestY = test.drop(["moneyness","price"], axis=1), test["price"]

In [10]:
# Normalize TestX
test_model_parameters = TestX.drop(["tau", "stockPrice", "strike"], axis=1)
test_option_properties = TestX.drop(["varsigma", "kappa", "delta", "v0", "rho"], axis=1)

normalized_test_model_parameters = pd.DataFrame()
for column in test_model_parameters.columns:
    normalized_test_model_parameters[column] = custom_min_max_normalization(
        test_model_parameters[column], 
        min_vals[column], 
        max_vals[column]
    )

# Fit the scaler with option_properties from the training data
scaler_option_properties = MinMaxScaler() 
scaler_option_properties.fit(option_properties)

# Transform the option_properties from the test data
normalized_test_option_properties = scaler_option_properties.transform(test_option_properties)
normalized_test_option_properties = pd.DataFrame(normalized_test_option_properties, columns=test_option_properties.columns)

TestX_normalized = pd.concat([normalized_test_model_parameters, normalized_test_option_properties], axis=1).values

# Now the scaler for Y
scaler_Y = MinMaxScaler()
scaler_Y.fit(Y.values.reshape(-1, 1))

# Normalize TestY
TestY_normalized = scaler_Y.transform(TestY.values.reshape(-1, 1))  # Use transform, not fit_transform
TestY_normalized = pd.Series(TestY_normalized.flatten(), name=TestY.name).values

In [11]:
# Define sequences of days
Testnum_features = TestX.shape[1]
Testnum_days = 53 # number of days
Testnum_samples = len(TestX)
Testnum_samples_per_day = Testnum_samples // Testnum_days

TestX_days = [TestX_normalized[i*Testnum_samples_per_day : (i+1)*Testnum_samples_per_day] for i in range(Testnum_days)]
TestY_days = [TestY_normalized[i*Testnum_samples_per_day : (i+1)*Testnum_samples_per_day] for i in range(Testnum_days)]

TestX_seq = np.stack(TestX_days, axis=1)
TestY_seq = np.stack(TestY_days, axis=1)

# Best LSTM

## 3 layers, nodes = 128, epochs = 50

In [13]:
# Data preprossesing -> X_train, X_test, X_val, Y_train, Y_test, Y_val
# Train - 80% dos dias, Test - 10% dos dias, Validation - 10% dos dias
train_size_per_day = int(0.8 * num_samples_per_day)
val_size_per_day = int(0.1 * num_samples_per_day)
test_size_per_day = num_samples_per_day - train_size_per_day - val_size_per_day

X_train = X_seq[:train_size_per_day]
Y_train = Y_seq[:train_size_per_day]

X_val = X_seq[train_size_per_day : train_size_per_day+val_size_per_day]
Y_val = Y_seq[train_size_per_day : train_size_per_day+val_size_per_day]

X_test = X_seq[train_size_per_day+val_size_per_day:]
Y_test = Y_seq[train_size_per_day+val_size_per_day:]

lstm = Sequential()
lstm.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
lstm.add(LSTM(units=128, return_sequences=True))
lstm.add(LSTM(units=128, return_sequences=True))
lstm.add(Dense(1,activation=tf.nn.relu))
lstm.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

# Define early stopping with more patience
early_stopping = EarlyStopping(monitor='val_loss', patience=7)

# Define learning rate scheduler
def scheduler(epoch, lr):
    if epoch != 0 and epoch % 20 == 0: 
        return lr * 0.5
    else:
        return lr
lr_scheduler = LearningRateScheduler(scheduler)

# Display the model summary
lstm.summary()

# Train the model
history = lstm.fit(X_train, Y_train,
                    validation_data=(X_val, Y_val),
                    epochs=50, batch_size=32, verbose=True, callbacks=[early_stopping, lr_scheduler])

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 53, 128)        │        70,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 53, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 53, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 53, 1)          │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 333,441 (1.27 MB)

 Trainable params: 333,441 (1.27 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 144s 170ms/step - loss: 0.0080 - val_loss: 2.3051e-04 - learning_rate: 0.0010
Epoch 2/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 135s 164ms/step - loss: 1.2127e-04 - val_loss: 1.8980e-04 - learning_rate: 0.0010
Epoch 3/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 122s 148ms/step - loss: 7.5849e-05 - val_loss: 6.4865e-05 - learning_rate: 0.0010
Epoch 4/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 118s 142ms/step - loss: 4.3638e-05 - val_loss: 1.0763e-04 - learning_rate: 0.0010
Epoch 5/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 125s 152ms/step - loss: 4.1253e-05 - val_loss: 4.1502e-05 - learning_rate: 0.0010
Epoch 6/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 127s 154ms/step - loss: 2.9398e-05 - val_loss: 5.0756e-05 - learning_rate: 0.0010
Epoch 7/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 142s 172ms/step - loss: 2.3164e-05 - val_loss: 3.3601e-05 - learning_rate: 0.0010
Epoch 8/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 125s 152ms/step - loss: 2.3211e-05 - val_loss: 4.2661e-05 - learning_rate: 0.0010
Epoch 9/50
825/825 ━━━━━━━━━

In [14]:
print("Evaluate: ", lstm.evaluate(X_test, Y_test))
pred = lstm.predict(X_test)
pred_denormalized = scaler_Y.inverse_transform(pred.flatten(order='F').reshape(-1,1))
Y_test_denormalized = scaler_Y.inverse_transform(Y_test.flatten(order='F').reshape(-1,1))
mse = mean_squared_error(Y_test_denormalized, pred_denormalized)
print("MSE desnormalized", mse)

print("Test Evaluate: ", lstm.evaluate(TestX_seq, TestY_seq))
Testpred = lstm.predict(TestX_seq).flatten(order='F').reshape(-1,1)
Testpred_denormalized = scaler_Y.inverse_transform(Testpred.flatten(order='F').reshape(-1,1))
TestY_denormalized = scaler_Y.inverse_transform(TestY_seq.flatten(order='F').reshape(-1,1))
mse = mean_squared_error(TestY_denormalized, Testpred_denormalized)
print("MSE desnormalized", mse)

104/104 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - loss: 7.0621e-07
Evaluate:  6.257074574023136e-07
104/104 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step
MSE desnormalized 0.00634218509225754
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 9.3053e-07
Test Evaluate:  1.3233300251158653e-06
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
MSE desnormalized 0.013405891023802389


## +50 epochs

In [27]:

lstm.compile(loss='mse', optimizer=Adam(learning_rate=2.5000e-04))

# Define early stopping with more patience
early_stopping = EarlyStopping(monitor='val_loss', patience=7)

# Define learning rate scheduler
def scheduler(epoch, lr):
    if epoch != 0 and epoch % 20 == 0: 
        return lr * 0.5
    else:
        return lr
lr_scheduler = LearningRateScheduler(scheduler)

# Display the model summary
lstm.summary()

# Train the model
history = lstm.fit(X_train, Y_train,
                    validation_data=(X_val, Y_val),
                    epochs=50, batch_size=32, verbose=True, callbacks=[early_stopping, lr_scheduler])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 53, 128)        │        70,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 53, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 53, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 53, 1)          │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 333,441 (1.27 MB)

 Trainable params: 333,441 (1.27 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 127s 134ms/step - loss: 8.9763e-07 - val_loss: 4.3339e-06 - learning_rate: 2.5000e-04
Epoch 2/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 101s 123ms/step - loss: 7.8681e-07 - val_loss: 3.8616e-06 - learning_rate: 2.5000e-04
Epoch 3/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 111s 134ms/step - loss: 8.5287e-07 - val_loss: 3.9967e-06 - learning_rate: 2.5000e-04
Epoch 4/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 108s 131ms/step - loss: 7.7131e-07 - val_loss: 4.0374e-06 - learning_rate: 2.5000e-04
Epoch 5/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 101s 122ms/step - loss: 1.0510e-06 - val_loss: 3.7068e-06 - learning_rate: 2.5000e-04
Epoch 6/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 101s 123ms/step - loss: 6.7749e-07 - val_loss: 3.3376e-06 - learning_rate: 2.5000e-04
Epoch 7/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 101s 122ms/step - loss: 6.3765e-07 - val_loss: 3.5370e-06 - learning_rate: 2.5000e-04
Epoch 8/50
825/825 ━━━━━━━━━━━━━━━━━━━━ 107s 130ms/step - loss: 8.5103e-07 - val_loss: 3.6769e-06 - learning_rate: 2.5

In [28]:
print("Evaluate: ", lstm.evaluate(X_test, Y_test))
pred = lstm.predict(X_test)
pred_denormalized = scaler_Y.inverse_transform(pred.flatten(order='F').reshape(-1,1))
Y_test_denormalized = scaler_Y.inverse_transform(Y_test.flatten(order='F').reshape(-1,1))
mse = mean_squared_error(Y_test_denormalized, pred_denormalized)
print("MSE desnormalized", mse)

print("Test Evaluate: ", lstm.evaluate(TestX_seq, TestY_seq))
Testpred = lstm.predict(TestX_seq).flatten(order='F').reshape(-1,1)
Testpred_denormalized = scaler_Y.inverse_transform(Testpred.flatten(order='F').reshape(-1,1))
TestY_denormalized = scaler_Y.inverse_transform(TestY_seq.flatten(order='F').reshape(-1,1))
mse = mean_squared_error(TestY_denormalized, Testpred_denormalized)
print("MSE desnormalized", mse)

104/104 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 4.9371e-07
Evaluate:  4.211565283185337e-07
104/104 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step
MSE desnormalized 0.004278467948630388
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 6.1460e-07
Test Evaluate:  8.34140280403517e-07
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
MSE desnormalized 0.008450194524307841


In [29]:
lstm.save('BestLSTM.keras')

# Results

In [12]:
lstm = load_model('BestLSTM.keras')

In [13]:
# Data preprossesing -> X_train, X_test, X_val, Y_train, Y_test, Y_val
# Train - 80% dos dias, Test - 10% dos dias, Validation - 10% dos dias
train_size_per_day = int(0.8 * num_samples_per_day)
val_size_per_day = int(0.1 * num_samples_per_day)
test_size_per_day = num_samples_per_day - train_size_per_day - val_size_per_day

X_train = X_seq[:train_size_per_day]
Y_train = Y_seq[:train_size_per_day]

X_val = X_seq[train_size_per_day : train_size_per_day+val_size_per_day]
Y_val = Y_seq[train_size_per_day : train_size_per_day+val_size_per_day]

X_test = X_seq[train_size_per_day+val_size_per_day:]
Y_test = Y_seq[train_size_per_day+val_size_per_day:]

In [14]:
train_predict = lstm.predict(X_train).flatten(order='F').reshape((-1,1))
val_predict = lstm.predict(X_val).flatten(order='F').reshape((-1,1))
test_predict = lstm.predict(X_test).flatten(order='F').reshape((-1,1))

825/825 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step


Training

In [15]:
mse = mean_squared_error(train_predict, Y_train.flatten(order='F').reshape((-1,1)))
mae = mean_absolute_error(train_predict, Y_train.flatten(order='F').reshape((-1,1)))
rmse = np.sqrt(mse)
r2 = r2_score(Y_train.flatten(order='F').reshape((-1,1)), train_predict)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)

Mean Squared Error (MSE): 4.653137991628704e-07
Mean Absolute Error (MAE): 0.000484927327098023
RMSE:  0.000682139134753952
R2:  0.9999840505462633


Validation

In [16]:
mse = mean_squared_error(val_predict, Y_val.flatten(order='F').reshape((-1,1)))
mae = mean_absolute_error(val_predict, Y_val.flatten(order='F').reshape((-1,1)))
rmse = np.sqrt(mse)
r2 = r2_score(Y_val.flatten(order='F').reshape((-1,1)), val_predict.flatten(order='F').reshape((-1,1)))
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)

Mean Squared Error (MSE): 2.843836275100394e-06
Mean Absolute Error (MAE): 0.0007160842624035654
RMSE:  0.001686367775753674
R2:  0.9998956943608741


Testing

In [17]:
mse = mean_squared_error(test_predict, Y_test.flatten(order='F').reshape((-1,1)))
mae = mean_absolute_error(test_predict, Y_test.flatten(order='F').reshape((-1,1)))
rmse = np.sqrt(mse)
r2 = r2_score(Y_test.flatten(order='F').reshape((-1,1)), test_predict.flatten(order='F').reshape((-1,1)))
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)

Mean Squared Error (MSE): 4.2233860782129907e-07
Mean Absolute Error (MAE): 0.00047450129855816116
RMSE:  0.0006498758403120546
R2:  0.9999854787696539


Test

In [18]:
num_iterations = 10

# List to store elapsed times
elapsed_times = []

for _ in range(num_iterations):
    # Start time
    start_time = time.time()

    # Code to measure
    testpredict = lstm.predict(TestX_seq).flatten(order='F').reshape((-1, 1))

    # End time
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time

    # Append elapsed time to list
    elapsed_times.append(elapsed_time)

# Calculate average elapsed time
average_elapsed_time = np.mean(elapsed_times)

print(f"Average execution time over {num_iterations} runs: {average_elapsed_time:.6f} seconds")

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Average execution time over 10 runs: 0.485283 seconds


In [19]:
testpredict = lstm.predict(TestX_seq).flatten(order='F').reshape((-1,1))

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [20]:
mse = mean_squared_error(testpredict, TestY_seq.flatten(order='F').reshape((-1,1)))
mae = mean_absolute_error(testpredict, TestY_seq.flatten(order='F').reshape((-1,1)))
rmse = np.sqrt(mse)
r2 = r2_score(TestY_seq.flatten(order='F').reshape((-1,1)), testpredict)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("RMSE: ", rmse)
print("R2: ", r2)

Mean Squared Error (MSE): 8.341399019076934e-07
Mean Absolute Error (MAE): 0.0005650430684754193
RMSE:  0.0009133125981325853
R2:  0.999963235151765


In [21]:
testpredict_denormalized = scaler_Y.inverse_transform(testpredict.flatten(order='F').reshape(-1, 1))
TestY_denormalized = scaler_Y.inverse_transform(TestY_seq.flatten(order='F').reshape(-1, 1))

mse_denormalized = mean_squared_error(TestY_denormalized, testpredict_denormalized)
mae_denormalized = mean_absolute_error(TestY_denormalized, testpredict_denormalized)
rmse_denormalized = np.sqrt(mse_denormalized)

print("MSE denormalized:", mse_denormalized)
print("MAE denormalized:", mae_denormalized)
print("RMSE denormalized:", rmse_denormalized)
print("R2 denormalized (same as normalized):", r2)

MSE denormalized: 0.008450189777806018
MAE denormalized: 0.05687158217300017
RMSE denormalized: 0.0919249138036366
R2 denormalized (same as normalized): 0.999963235151765
